# import libraries and fns

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import pandas as pd
import time
import re
import json

## Dataframe fields overview to scrape

In [25]:
property_df

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,...,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,href,Price


In [53]:


def extract_data(soup, id_column,var_names,id_name='ad_id'):
# Find the script tag that contains the JSON data
    if soup.find('script', string=lambda s: '({"experiments":' in s):
        script_tag = soup.find('script', string=lambda s: '({"experiments":' in s)
    else:
        raise ValueError("Didnt find any data.. error!")
        return 'idk why'
    if script_tag:
        # Extract the JavaScript code from the script tag
        js_code = script_tag.string

        # Use a regular expression to extract the JSON data
        match = re.search(r'({"experiments":.+?})\);', js_code)

        if match:
            json_data = match.group(1)

            # Load the JSON data
            data = json.loads(json_data)
            variables = {}
            
            #check if unique
            variables['ad_id'] = data.get(id_name)
            #print(sum(id_column.values==ad_id),' hi')
            if id_column.str.count(variables['ad_id']).sum() != 0:
                return 'Ad already recorded'
            #if sum(id_column.values==ad_id) != 0:
            #    return 'Ad already recorded'
            variables['href'] = soup.find('link', {'rel': 'canonical'}).get('href')[len(baseurl):]
            
            
            variable_names = [i for i in var_names if i not in ['href','creation_date','description_section']]
            
            for i in variable_names:
                if i =='area':
                    variables[i] = data.get('ft')
                elif i =='city':
                    variables[i] = data.get('loc_1_name').strip(':;')
                
                else:
                    variables[i]=data.get(i)
                
                
            # Extract relevant information
#             price = data.get('price')
#             num_rooms = data.get('rooms')
#             num_bathrooms = data.get('bathrooms')
#             location = data.get('loc_name')
#             ad_title = data.get('ad_title')
#             seller_type = data.get('seller_type')

#             area_unit = data.get('area_unit')
#             currency_unit = data.get('currency_unit')
#             payment_option = data.get('payment_option')
#             number_of_photos = data.get('number_of_photos')
#             construction_state = data.get('construction_state')
#             package_type = data.get('package_type')
#             down_payment = data.get('down_payment')
            
#             country = data.get('loc_1_name').strip(':;')  # Clean the 'Country' field
#             area = data.get('ft')  # Assuming 'ft' contains area information
#             ownership = data.get('ownership')  # Assuming 'ownership' contains ownership information
#             Furnished = data.get('furnished')
            
            creation_date_span = soup2.find('span', {'aria-label': 'Creation date'})
            if creation_date_span:
                variables['creation_date'] = creation_date_span.get_text(strip=True)
                
            amenities_section = soup2.find('div', {'aria-label': 'Features'})
            amenities_list = []
            # Check if the amenities section is found
            if amenities_section:
                # Find all spans within the amenities section
                amenities_spans = amenities_section.find_all('span')

                # Extract the text from each span to get the list of amenities
                variables['amenities_list'] = [span.get_text(strip=True) for span in amenities_spans]
                
            description_section = soup.find('div', {'aria-label': 'Description'})

            # Check if the description section is found
            if description_section:
                # Extract the text from the description section
                variables['description_text'] = description_section.get_text(strip=True).replace('\n','').replace('\t','')

            # Create a new row
#             new_row = pd.DataFrame([[ad_id, ad_title, country, location, num_rooms, num_bathrooms, area,Furnished, creation_date,\
#                                      ownership, seller_type, amenities_list,description_text,\
#                                      area_unit,currency_unit, payment_option,number_of_photos,\
#                                      construction_state,package_type,down_payment,href,price]], columns=columns)
            new_row = pd.DataFrame([variables])


#             new_row = pd.DataFrame([[ad_id, ad_title, country, location, num_rooms, num_bathrooms, area,Furnished, creation_date,\
#                                      ownership, seller_type, amenities_list,description_text,\
#                                      area_unit,currency_unit, payment_option,number_of_photos,\
#                                      construction_state,package_type,down_payment,price]], columns=columns)

            # Append the new row to the existing dataframe
            #property_df = property_df.append(new_row, ignore_index=True)

            # Print the updated dataframe
            #display(property_df)
            return new_row
            

        else:
            return 'JSON data not found in the script tag.'


# define variables, pages and urls

In [54]:
baseurl = 'https://www.dubizzle.com.eg'

In [55]:
trial = 'https://www.dubizzle.com.eg/en/properties/apartments-duplex-for-sale/?page=3'

In [56]:
apartments_page = 'https://www.dubizzle.com.eg/en/properties/apartments-duplex-for-sale/?page='

In [57]:
apartments_page = 'https://www.dubizzle.com.eg/en/properties/apartments-duplex-for-sale/new-cairo/?page='

In [58]:
 headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'  
    }

In [59]:
columns = ['ad_id', 'ad_title', 'city', 'loc_name', 'rooms', 'bathrooms', 'area','furnished'\
           ,'creation_date', 'ownership', 'seller Type','amenities_list','description_text',\
           'area_unit','currency_unit', 'payment_option','number_of_photos' ,'construction_state','package_type','down_payment',\
           'href',
           'price']
property_df = pd.DataFrame(columns=columns)

In [10]:
# columns = ['Ad ID', 'Ad Title', 'Country', 'Location', 'Number of Rooms', 'Number of Bathrooms', 'Area','Furnished'\
#            ,'creation_date', 'Ownership', 'Seller Type','amenities_list','description_text',\
#            'area_unit','currency_unit', 'payment_option','number_of_photos' ,'construction_state','package_type','down_payment',\
#            'href',
#            'Price']
# property_df = pd.DataFrame(columns=columns)

In [20]:
property_df

,ad_id,ad_title,city,loc_name,rooms,bathrooms,ft,furnished,creation_date,ownership,...,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,href,price,area
0,199890864,فورى بنتهاوس متشطب بحرى بالتقسيط فيفث سكوير fi...,None,Fifth Square,3,3,NaN,None,42 minutes ago,resale,...,Sq. M.,EGP,None,20,1,None,None,/en/ad/%D9%81%D9%88%D8%B1%D9%89-%D8%A8%D9%86%D...,14500000,223.0
1,199349683,new cairo beit al watan - شقة للبيع 202م بيت ا...,None,Beit Al Watan,3,3,NaN,2,43 minutes ago,primary,...,Sq. M.,EGP,2,8,4,None,1173300,/en/ad/new-cairo-beit-al-watan-%D8%B4%D9%82%D8...,3911000,202.0


In [60]:
property_df = pd.read_csv('olx_data_newcairo_2.7k_21_12.csv')
property_df.drop('Unnamed: 0',axis=1,inplace=True)
property_df['Ad ID'] = property_df['Ad ID'].astype('str')

In [61]:
property_df['Country'].value_counts(dropna=False)

Cairo    2763
Name: Country, dtype: int64

In [62]:
property_df.columns = columns

In [63]:
property_df.head()

,ad_id,ad_title,city,loc_name,rooms,bathrooms,area,furnished,creation_date,ownership,...,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,href,price
0,199789460,studio in Mivida blue views prime location fro...,Cairo,New Cairo,1,1,86,1.0,18 minutes ago,resale,...,Descriptionstudio in Mivida blue views prime l...,Sq. M.,EGP,1.0,2,NaN,NaN,5100000.0,/en/ad/studio-in-mivida-blue-views-prime-locat...,5100000
1,199703703,دوبلكس للبيع 260م باقساط 8 سنوات في التجمع الخ...,Cairo,Eelaf Residence,5,5,260,2.0,18 minutes ago,primary,...,Descriptionكمبوند ايلاف - التجمع الخامس دوبلكس...,Sq. M.,EGP,2.0,10,NaN,NaN,1143000.0,/en/ad/%D8%AF%D9%88%D8%A8%D9%84%D9%83%D8%B3-%D...,11428000
2,199831199,شقه للبيع باللوتس 180م امام مول Rock gold اكبر...,Cairo,El Lotus,3,2,180,2.0,34 minutes ago,primary,...,Descriptionشقه للبيع باللوتساول سكن 180م دور ر...,Sq. M.,EGP,1.0,16,1.0,NaN,NaN,/en/ad/%D8%B4%D9%82%D9%87-%D9%84%D9%84%D8%A8%D...,4000000
3,199788142,شقة متشطبة بالكامل للبيع ف التجمع عند الجامعه ...,Cairo,New Cairo,3,2,180,1.0,18 minutes ago,primary,...,Descriptionكمـبوند ذا ادريس ايست التجمـع الخـا...,Sq. M.,EGP,3.0,5,1.0,NaN,1678000.0,/en/ad/%D8%B4%D9%82%D8%A9-%D9%85%D8%AA%D8%B4%D...,1678000
4,197283683,بنتهاوس 241م + روف 132م بيت الوطن بالقرب الحي ...,Cairo,New Cairo,4,4,241,2.0,18 minutes ago,primary,...,Descriptionاسكن في افضل احياء بيت الوطن علي شا...,Sq. M.,EGP,2.0,9,4.0,NaN,1091875.0,/en/ad/%D8%A8%D9%86%D8%AA%D9%87%D8%A7%D9%88%D8...,4367500


# inital logic

In [14]:
# r = requests.get(trial)
# soup = BeautifulSoup(r.content, 'lxml')
# productlist = soup.find_all('li', class_='undefined')

In [15]:
# for item in productlist:
#     #for link in item.find_all('title', class_='undefined'):
#         for link in item.find_all('a',href=True):#, class_='undefined'))
#             print(link["href"])
#             print('hi')
#             ref_url.append(link["href"])

# merge and loop logic

## Scraping page per ad and extract required data, merge to dataframe

In [64]:
#loading pages 2 to 5
for i in range(2,600):
    time.sleep(1)
    ref_url = []
    page = apartments_page + str(i)
    
    max_retries = 3  # Number of maximum retry attempts
    retry_count = 0
    while retry_count < max_retries:
        try:
            r = requests.get(page)
            print('Loading page ',i)
            soup = BeautifulSoup(r.content, 'lxml')
            productlist = soup.find_all('li', class_='undefined')
            for item in productlist:
                #for link in item.find_all('title', class_='undefined'):
                    for link in item.find_all('a',href=True):#, class_='undefined'))
                        #print(link["href"])
                        #print('hi')
                        ref_url.append(link["href"])
            
            ref_url = list(set(ref_url))
            if len(ref_url)==0:
                print("No ads found. Retrying...")
                retry_count += 1
                time.sleep(200)
                continue
            ref_url_found = list(property_df[property_df.href.isin(ref_url)].href)
            ref_url = [string for string in ref_url if string not in ref_url_found]
            #print(ref_url)
            print(len(list(set(ref_url))),' ads loaded')
            print('Links for page ',i,' loaded.')
 
            
            break
        except Exception as e:
            # Handle the specific exception that occurred
            print(f"Exception: {e}")
            # Optionally, you can add a delay before retrying
            time.sleep(200)
            retry_count += 1
            continue
   

    
    for number,ref_link in enumerate(ref_url):
        
        print('ad number ',number+1,end=' ;')#,ref_link)
        time.sleep(2)# suspend execution for 2 secs
        max_retries = 3  # Number of maximum retry attempts
        retry_count = 0
        while retry_count < max_retries:

            try:
                r2 = requests.get(baseurl+ref_link)
                soup2 = BeautifulSoup(r2.content, 'lxml')
                new_row = extract_data(soup2,property_df['ad_id'],columns)
                if not isinstance(new_row, str):
                    property_df = pd.concat([property_df,new_row], ignore_index=True)
                    print('Finished succesfully')
                else:
                    print(new_row)
                break
            except Exception as e:
                # Handle the specific exception that occurred
                print(f"Exception: {e}")
                # Optionally, you can add a delay before retrying
                time.sleep(200)
                retry_count += 1
                continue   
        #s = 'https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D8%A9-%D9%85%D9%81%D8%B1%D9%88%D8%B4%D8%A9-%D8%AC%D8%AF%D9%8A%D8%AF-%D8%A7%D9%88%D9%84-%D8%B3%D9%83%D9%86-%D9%A2%D9%A5%D9%A0%D9%85-%D8%A8%D8%A7%D9%84%D8%AD%D9%8A%D9%A9-ID199712680.html'
        #r2 = requests.get(s)

        #property_df
          
    

Loading page  2
48  ads loaded
Links for page  2  loaded.
ad number  1 ;Finished succesfully
ad number  2 ;Finished succesfully
ad number  3 ;Finished succesfully
ad number  4 ;Finished succesfully
ad number  5 ;Finished succesfully
ad number  6 ;Finished succesfully
ad number  7 ;Finished succesfully
ad number  8 ;Finished succesfully
ad number  9 ;Finished succesfully
ad number  10 ;Finished succesfully
ad number  11 ;Finished succesfully
ad number  12 ;Finished succesfully
ad number  13 ;Finished succesfully
ad number  14 ;Finished succesfully
ad number  15 ;Finished succesfully
ad number  16 ;Finished succesfully
ad number  17 ;Finished succesfully
ad number  18 ;Finished succesfully
ad number  19 ;Finished succesfully
ad number  20 ;Finished succesfully
ad number  21 ;Finished succesfully
ad number  22 ;Finished succesfully
ad number  23 ;Finished succesfully
ad number  24 ;Finished succesfully
ad number  25 ;Finished succesfully
ad number  26 ;Finished succesfully
ad number  27 ;

KeyboardInterrupt: 

In [68]:
property_df#.isna().sum()

,ad_id,ad_title,city,loc_name,rooms,bathrooms,area,furnished,creation_date,ownership,...,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,href,price
0,199789460,studio in Mivida blue views prime location fro...,Cairo,New Cairo,1,1,86,1.0,18 minutes ago,resale,...,Descriptionstudio in Mivida blue views prime l...,Sq. M.,EGP,1.0,2,NaN,NaN,5100000.0,/en/ad/studio-in-mivida-blue-views-prime-locat...,5100000
1,199703703,دوبلكس للبيع 260م باقساط 8 سنوات في التجمع الخ...,Cairo,Eelaf Residence,5,5,260,2.0,18 minutes ago,primary,...,Descriptionكمبوند ايلاف - التجمع الخامس دوبلكس...,Sq. M.,EGP,2.0,10,NaN,NaN,1143000.0,/en/ad/%D8%AF%D9%88%D8%A8%D9%84%D9%83%D8%B3-%D...,11428000
2,199831199,شقه للبيع باللوتس 180م امام مول Rock gold اكبر...,Cairo,El Lotus,3,2,180,2.0,34 minutes ago,primary,...,Descriptionشقه للبيع باللوتساول سكن 180م دور ر...,Sq. M.,EGP,1.0,16,1.0,NaN,NaN,/en/ad/%D8%B4%D9%82%D9%87-%D9%84%D9%84%D8%A8%D...,4000000
3,199788142,شقة متشطبة بالكامل للبيع ف التجمع عند الجامعه ...,Cairo,New Cairo,3,2,180,1.0,18 minutes ago,primary,...,Descriptionكمـبوند ذا ادريس ايست التجمـع الخـا...,Sq. M.,EGP,3.0,5,1.0,NaN,1678000.0,/en/ad/%D8%B4%D9%82%D8%A9-%D9%85%D8%AA%D8%B4%D...,1678000
4,197283683,بنتهاوس 241م + روف 132م بيت الوطن بالقرب الحي ...,Cairo,New Cairo,4,4,241,2.0,18 minutes ago,primary,...,Descriptionاسكن في افضل احياء بيت الوطن علي شا...,Sq. M.,EGP,2.0,9,4.0,NaN,1091875.0,/en/ad/%D8%A8%D9%86%D8%AA%D9%87%D8%A7%D9%88%D8...,4367500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2793,500368402,"Apartment for sale, two bedroom, 130m with a g...",Cairo,Taj City,2,2,130,2,5 minutes ago,primary,...,DescriptionTaj city is characterized by its pr...,Sq. M.,EGP,None,3,None,free,NaN,/en/ad/apartment-for-sale-two-bedroom-130m-wit...,4119000
2794,500396549,"Apartment in Rehab City, special finishes",Cairo,Rehab City,3,3,167,2,17 minutes ago,resale,...,DescriptionApartment 167m for sale in Rehab Ci...,Sq. M.,EGP,None,9,None,free,NaN,/en/ad/apartment-in-rehab-city-special-finishe...,6200000
2795,199876311,بجوار حديقه الأسرة شقة لقطة 80 م يلو على طريق ...,Cairo,New Cairo,1,1,80,None,1 hour ago,primary,...,Description:- تفاصيل الوحدة - مساحة : 80 متر(غ...,Sq. M.,EGP,3,6,None,None,NaN,/en/ad/%D8%A8%D8%AC%D9%88%D8%A7%D8%B1-%D8%AD%D...,400000
2796,199928086,للبيع شقة بتشطيب كامل بالتكييفات ف كمبوند زد ا...,Cairo,Zed East,3,2,165,None,35 minutes ago,resale,...,Descriptionللبيع شقة 165م متشطبه بالتكييفات و ...,Sq. M.,EGP,2,13,1,None,5.2,/en/ad/%D9%84%D9%84%D8%A8%D9%8A%D8%B9-%D8%B4%D...,9000000


In [27]:
new_row#.values

,ad_id,href,ad_title,city,loc_name,rooms,bathrooms,area,furnished,ownership,...,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,price,creation_date
0,199349683,/en/ad/new-cairo-beit-al-watan-%D8%B4%D9%82%D8...,new cairo beit al watan - شقة للبيع 202م بيت ا...,None,Beit Al Watan,3,3,202,2,primary,...,Descriptionامتلك شقتك دلوقتي بارقى مناطق التجم...,Sq. M.,EGP,2,8,4,None,1173300,3911000,43 minutes ago


In [42]:
#loading pages 2 to 5
for i in range(2,600):
    time.sleep(1)
    ref_url = []
    page = apartments_page + str(i)
    
    max_retries = 3  # Number of maximum retry attempts
    retry_count = 0
    while retry_count < max_retries:
        try:
            r = requests.get(page)
            print('Loading page ',i)
            soup = BeautifulSoup(r.content, 'lxml')
            productlist = soup.find_all('li', class_='undefined')
            for item in productlist:
                #for link in item.find_all('title', class_='undefined'):
                    for link in item.find_all('a',href=True):#, class_='undefined'))
                        #print(link["href"])
                        #print('hi')
                        ref_url.append(link["href"])
            
            ref_url = list(set(ref_url))
            if len(ref_url)==0:
                print("No ads found. Retrying...")
                retry_count += 1
                time.sleep(200)
                continue
            ref_url_found = list(property_df[property_df.href.isin(ref_url)].href)
            ref_url = [string for string in ref_url if string not in ref_url_found]
            #print(ref_url)
            print(len(list(set(ref_url))),' ads loaded')
            print('Links for page ',i,' loaded.')
 
            
            break
        except Exception as e:
            # Handle the specific exception that occurred
            print(f"Exception: {e}")
            # Optionally, you can add a delay before retrying
            time.sleep(200)
            retry_count += 1
            continue
   

    
    for number,ref_link in enumerate(ref_url):
        
        print('ad number ',number+1,end=' ;')#,ref_link)
        time.sleep(2)# suspend execution for 2 secs
        max_retries = 3  # Number of maximum retry attempts
        retry_count = 0
        while retry_count < max_retries:

            try:
                r2 = requests.get(baseurl+ref_link)
                soup2 = BeautifulSoup(r2.content, 'lxml')
                new_row = extract_data(soup2,property_df['Ad ID'],columns)
                if not isinstance(new_row, str):
                    property_df = property_df.append(new_row, ignore_index=True)
                    print('Finished succesfully')
                else:
                    print(new_row)
                break
            except Exception as e:
                # Handle the specific exception that occurred
                print(f"Exception: {e}")
                # Optionally, you can add a delay before retrying
                time.sleep(200)
                retry_count += 1
                continue   
        #s = 'https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D8%A9-%D9%85%D9%81%D8%B1%D9%88%D8%B4%D8%A9-%D8%AC%D8%AF%D9%8A%D8%AF-%D8%A7%D9%88%D9%84-%D8%B3%D9%83%D9%86-%D9%A2%D9%A5%D9%A0%D9%85-%D8%A8%D8%A7%D9%84%D8%AD%D9%8A%D9%A9-ID199712680.html'
        #r2 = requests.get(s)

        #property_df
          
    

Loading page  2
50  ads loaded
Links for page  2  loaded.
ad number  1 ;

KeyboardInterrupt: 

In [19]:
new_row

,ad_id,href,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,...,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,Price,creation_date
0,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
1,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
2,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
3,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
4,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
5,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
6,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
7,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
8,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago
9,199925608,/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D9%84%D...,None,None,None,None,None,None,None,None,...,Descriptionفرصه لن تتكرر للبيع في الرحاب بتشطي...,Sq. M.,EGP,1,20,1,featured,None,None,1 hour ago


In [ ]:
property_df

In [18]:
#property_df.to_csv('olx_data_newcairo_5.8k_24_12.csv')

In [ ]:
#loading pages 2 to 5
for i in range(141,600):
    time.sleep(1)
    ref_url = []
    page = apartments_page + str(i)
    
    max_retries = 3  # Number of maximum retry attempts
    retry_count = 0
    while retry_count < max_retries:
        try:
            r = requests.get(page)
            print('Loading page ',i)
            soup = BeautifulSoup(r.content, 'lxml')
            productlist = soup.find_all('li', class_='undefined')
            for item in productlist:
                #for link in item.find_all('title', class_='undefined'):
                    for link in item.find_all('a',href=True):#, class_='undefined'))
                        #print(link["href"])
                        #print('hi')
                        ref_url.append(link["href"])
            
            ref_url = list(set(ref_url))
            if len(ref_url)==0:
                print("No ads found. Retrying...")
                retry_count += 1
                time.sleep(200)
                continue
            ref_url_found = list(property_df[property_df.href.isin(ref_url)].href)
            ref_url = [string for string in ref_url if string not in ref_url_found]
            #print(ref_url)
            print(len(list(set(ref_url))),' ads loaded')
            print('Links for page ',i,' loaded.')

            
            break
        except Exception as e:
            # Handle the specific exception that occurred
            print(f"Exception: {e}")
            # Optionally, you can add a delay before retrying
            time.sleep(200)
            retry_count += 1
            continue
   

    
    for number,ref_link in enumerate(ref_url):
        
        print('ad number ',number+1,end=' ;')#,ref_link)
        time.sleep(2)# suspend execution for 2 secs
        max_retries = 3  # Number of maximum retry attempts
        retry_count = 0
        while retry_count < max_retries:

            try:
                r2 = requests.get(baseurl+ref_link)
                soup2 = BeautifulSoup(r2.content, 'lxml')
                new_row = extract_data(soup2,property_df['Ad ID'])
                if not isinstance(new_row, str):
                    property_df = property_df.append(new_row, ignore_index=True)
                    print('Finished succesfully')
                else:
                    print(new_row)
                break
            except Exception as e:
                # Handle the specific exception that occurred
                print(f"Exception: {e}")
                # Optionally, you can add a delay before retrying
                time.sleep(200)
                retry_count += 1
                continue   
        #s = 'https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D8%A9-%D9%85%D9%81%D8%B1%D9%88%D8%B4%D8%A9-%D8%AC%D8%AF%D9%8A%D8%AF-%D8%A7%D9%88%D9%84-%D8%B3%D9%83%D9%86-%D9%A2%D9%A5%D9%A0%D9%85-%D8%A8%D8%A7%D9%84%D8%AD%D9%8A%D9%A9-ID199712680.html'
        #r2 = requests.get(s)

        #property_df
          
    

Loading page  141
34  ads loaded
Links for page  141  loaded.
ad number  1 ;Finished succesfully
ad number  2 ;Finished succesfully
ad number  3 ;Finished succesfully
ad number  4 ;Finished succesfully
ad number  5 ;Finished succesfully
ad number  6 ;Finished succesfully
ad number  7 ;Finished succesfully
ad number  8 ;Finished succesfully
ad number  9 ;Finished succesfully
ad number  10 ;Finished succesfully
ad number  11 ;Finished succesfully
ad number  12 ;Finished succesfully
ad number  13 ;Finished succesfully
ad number  14 ;Finished succesfully
ad number  15 ;Finished succesfully
ad number  16 ;Finished succesfully
ad number  17 ;Finished succesfully
ad number  18 ;Finished succesfully
ad number  19 ;Finished succesfully
ad number  20 ;Finished succesfully
ad number  21 ;Finished succesfully
ad number  22 ;Finished succesfully
ad number  23 ;Finished succesfully
ad number  24 ;Finished succesfully
ad number  25 ;Finished succesfully
ad number  26 ;Finished succesfully
ad number  

In [40]:
for i in []:
    print(i)

In [39]:
r.content

b'<!DOCTYPE html><html lang="en" dir="ltr" itemscope="" itemType="http://schema.org/WebPage"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=0"/><link rel="dns-prefetch" href="https://ll8iz711cs-dsn.algolia.net"/><link rel="dns-prefetch" href="https://www.googletagmanager.com"/><link rel="dns-prefetch" href="https://www.google-analytics.com"/><link rel="dns-prefetch" href="https://images.bayut.com"/><link rel="apple-touch-icon" sizes="180x180" href="/assets/apple-touch-icon.318a683f5e331c46c23eb3742a9f3d49.png"/><link rel="icon" type="image/png" sizes="16x16" href="/assets/favicon-16x16.771c69f9ab365d2b39ca63a11a5edc57.png"/><link rel="icon" type="image/png" sizes="32x32" href="/assets/favicon-32x32.db52812416fd3c125e3502aabf54bacf.png"/><link rel="manifest" href="/assets/66a760759488eba1b84b5b3cd7e2ceed.json"/><link rel="mask-icon" href="/assets/safari-pinned-tab.2b2cbe8d955838bf7bf59d8142e689df.svg" color="#28b16d"/><me

In [ ]:



# Find the script tag that contains the JSON data
if soup2.find('script', string=lambda s: '({"experiments":' in s):
    script_tag = soup2.find('script', string=lambda s: '({"experiments":' in s)
else:
    raise ValueError("Didnt find any data.. error!")
    print('idk why')
if script_tag:
    # Extract the JavaScript code from the script tag
    js_code = script_tag.string

    # Use a regular expression to extract the JSON data
    match = re.search(r'({"experiments":.+?})\);', js_code)

    if match:
        json_data = match.group(1)

        # Load the JSON data
        data = json.loads(json_data)
        variables = {}

        #check if unique
        variables['ad_id'] = data.get(id_name)
        #print(sum(id_column.values==ad_id),' hi')
        if property_df['ad_id'].str.count(variables['ad_id']).sum() != 0:
            print('Ad already recorded')
        #if sum(id_column.values==ad_id) != 0:
        #    return 'Ad already recorded'
        variables['href'] = soup.find('link', {'rel': 'canonical'}).get('href')[len(baseurl):]


        variable_names = [i for i in var_names if i not in ['ad_id','href','creation_date','description_section']]

        for i in variable_names:
            if i =='area':
                variables[i] = data.get('ft')
            elif i =='city':
                variables[i] = data.get('loc_1_name').strip(':;')

            else:
                variables[i]=data.get(i)


        # Extract relevant information
#             price = data.get('price')
#             num_rooms = data.get('rooms')
#             num_bathrooms = data.get('bathrooms')
#             location = data.get('loc_name')
#             ad_title = data.get('ad_title')
#             seller_type = data.get('seller_type')

#             area_unit = data.get('area_unit')
#             currency_unit = data.get('currency_unit')
#             payment_option = data.get('payment_option')
#             number_of_photos = data.get('number_of_photos')
#             construction_state = data.get('construction_state')
#             package_type = data.get('package_type')
#             down_payment = data.get('down_payment')

#             country = data.get('loc_1_name').strip(':;')  # Clean the 'Country' field
#             area = data.get('ft')  # Assuming 'ft' contains area information
#             ownership = data.get('ownership')  # Assuming 'ownership' contains ownership information
#             Furnished = data.get('furnished')

        creation_date_span = soup2.find('span', {'aria-label': 'Creation date'})
        if creation_date_span:
            variables['creation_date'] = creation_date_span.get_text(strip=True)

        amenities_section = soup2.find('div', {'aria-label': 'Features'})
        amenities_list = []
        # Check if the amenities section is found
        if amenities_section:
            # Find all spans within the amenities section
            amenities_spans = amenities_section.find_all('span')

            # Extract the text from each span to get the list of amenities
            variables['amenities_list'] = [span.get_text(strip=True) for span in amenities_spans]

        description_section = soup.find('div', {'aria-label': 'Description'})

        # Check if the description section is found
        if description_section:
            # Extract the text from the description section
            variables['description_text'] = description_section.get_text(strip=True).replace('\n','').replace('\t','')

        # Create a new row
#             new_row = pd.DataFrame([[ad_id, ad_title, country, location, num_rooms, num_bathrooms, area,Furnished, creation_date,\
#                                      ownership, seller_type, amenities_list,description_text,\
#                                      area_unit,currency_unit, payment_option,number_of_photos,\
#                                      construction_state,package_type,down_payment,href,price]], columns=columns)
        new_row = pd.DataFrame([variables])


#             new_row = pd.DataFrame([[ad_id, ad_title, country, location, num_rooms, num_bathrooms, area,Furnished, creation_date,\
#                                      ownership, seller_type, amenities_list,description_text,\
#                                      area_unit,currency_unit, payment_option,number_of_photos,\
#                                      construction_state,package_type,down_payment,price]], columns=columns)

        # Append the new row to the existing dataframe
        #property_df = property_df.append(new_row, ignore_index=True)

        # Print the updated dataframe

    else:
        print('JSON data not found in the script tag.')


In [ ]:
data

In [46]:
property_df.to_csv('olx_data_9.6k_21_12.csv')

In [97]:
r2

<Response [200]>

In [90]:
ref_link

'/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%81%D9%8A-%D8%A7%D9%84%D8%B2%D9%82%D8%A7%D8%B2%D9%8A%D9%82-%D8%A8%D8%A3%D9%81%D8%B6%D9%84-%D8%A7%D9%84%D9%85%D9%88%D8%A7%D9%82%D8%B9-%D9%88%D9%85%D8%B3%D8%AC%D9%84%D8%A9-%D8%A8%D8%A7%D9%84%D8%B4%D9%87%D8%B1-%D8%A7%D9%84%D8%B9%D9%82%D8%A7%D8%B1%D9%8A-%D8%AA%D8%B5%D9%84%D8%AD-%D8%AA%D9%85%D9%88%D9%8A%D9%84-%D8%B9%D9%82%D8%A7%D8%B1%D9%8A-ID199791357.html'

In [91]:
ref_url[33]

'/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%81%D9%8A-%D8%A7%D9%84%D8%B2%D9%82%D8%A7%D8%B2%D9%8A%D9%82-%D8%A8%D8%A3%D9%81%D8%B6%D9%84-%D8%A7%D9%84%D9%85%D9%88%D8%A7%D9%82%D8%B9-%D9%88%D9%85%D8%B3%D8%AC%D9%84%D8%A9-%D8%A8%D8%A7%D9%84%D8%B4%D9%87%D8%B1-%D8%A7%D9%84%D8%B9%D9%82%D8%A7%D8%B1%D9%8A-%D8%AA%D8%B5%D9%84%D8%AD-%D8%AA%D9%85%D9%88%D9%8A%D9%84-%D8%B9%D9%82%D8%A7%D8%B1%D9%8A-ID199791357.html'

In [86]:
1

1

In [83]:
property_df4 = property_df.copy()

In [108]:
property_df4

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,...,amenities_list,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,Price
0,500377775,Buy in Taj City in front of Cairo Airport,Cairo,Taj City,4,3,208,None,50 minutes ago,primary,...,"[Security, Pool, Natural Gas, Landline]",Descriptionاشتري في تاج سيتي امام مطار القاهرة...,Sq. M.,EGP,None,8,None,free,None,1099000
1,198819436,امتلك شقه فخمه في أيون تاورز من مراكز الشيخ زا...,Giza,Aeon,3,3,246,None,15 minutes ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionامتلك فورا شقه فندقيه راقيه من مراك...,Sq. M.,EGP,3,10,1,None,None,18000000
2,199745354,شقة 130م بجاردن بخصم كاش 39% في كمبوند تاج سيت...,Cairo,Taj City,2,3,130,None,4 days ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقة ارضي بجاردن 131م متر للبيع في ت...,Sq. M.,EGP,3,7,4,elite,645700,6457000
3,199765010,استلم فوري ب 200 الف ستديو لقطه للبيع في صن كا...,Giza,Sun Capital,1,1,56,2,21 minutes ago,primary,...,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionاستلام فوري ب 10% بقلب اكتوبر اقساط...,Sq. M.,EGP,3,11,2,featured,200000,200000
4,199079978,شقه للبيع فى 6 أكتوبر Badya Palm Hills,Giza,Badya Palm Hills,3,2,155,2,1 month ago,primary,...,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقه للبيع بكمبوند باديا بمقدم 600 ا...,Sq. M.,EGP,3,12,1,None,600000,600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2254,199784380,دوبليكس للبيع كمبوند حدائق المهندسين مرحله 2 ا...,Giza,Zayed 2000,3,3,185,2,12 hours ago,resale,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionدوبليكس للبيع كمبوند حدائق المهندسي...,Sq. M.,EGP,1,10,1,None,None,6700000
2255,199784266,للبيع شقه 162متر في تاج سيتي مرحله فيلات امام ...,Cairo,Taj City,3,2,162,None,12 hours ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقه3غرف للبيع في تاج_سيتي علي طريق ...,Sq. M.,EGP,3,1,1,None,None,765000
2256,199784368,للبيع بأفضل سعر شقه 120 م + جاردن 105 م بفيو م...,Cairo,New Cairo,2,2,120,None,12 hours ago,primary,...,"[Private Garden, Security, Covered Parking, Ma...",Descriptionبأفضل سعر شقة 120م غرفتين للبيع في ...,Sq. M.,EGP,3,5,None,None,None,420000
2257,199784329,شقة للبيع بالجولدن سكوير علي بعد دقائق من كل ا...,Cairo,5th Settlement,3,3,184,2,12 hours ago,primary,...,"[Balcony, Private Garden, Security, Covered Pa...",Descriptionشقة بكمبوند Cattalya مساحة الشقة 18...,Sq. M.,EGP,3,3,1,None,None,10000000


In [109]:
property_df4.to_csv('olx_2.2krows.csv')

In [78]:
property_df3 = property_df.copy()

In [80]:
property_df3

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,...,amenities_list,description_text,area_unit,currency_unit,payment_option,number_of_photos,construction_state,package_type,down_payment,Price
0,500377775,Buy in Taj City in front of Cairo Airport,Cairo,Taj City,4,3,208,None,50 minutes ago,primary,...,"[Security, Pool, Natural Gas, Landline]",Descriptionاشتري في تاج سيتي امام مطار القاهرة...,Sq. M.,EGP,None,8,None,free,None,1099000
1,198819436,امتلك شقه فخمه في أيون تاورز من مراكز الشيخ زا...,Giza,Aeon,3,3,246,None,15 minutes ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionامتلك فورا شقه فندقيه راقيه من مراك...,Sq. M.,EGP,3,10,1,None,None,18000000
2,199745354,شقة 130م بجاردن بخصم كاش 39% في كمبوند تاج سيت...,Cairo,Taj City,2,3,130,None,4 days ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقة ارضي بجاردن 131م متر للبيع في ت...,Sq. M.,EGP,3,7,4,elite,645700,6457000
3,199765010,استلم فوري ب 200 الف ستديو لقطه للبيع في صن كا...,Giza,Sun Capital,1,1,56,2,21 minutes ago,primary,...,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionاستلام فوري ب 10% بقلب اكتوبر اقساط...,Sq. M.,EGP,3,11,2,featured,200000,200000
4,199079978,شقه للبيع فى 6 أكتوبر Badya Palm Hills,Giza,Badya Palm Hills,3,2,155,2,1 month ago,primary,...,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقه للبيع بكمبوند باديا بمقدم 600 ا...,Sq. M.,EGP,3,12,1,None,600000,600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,199756559,شقة بكمبوند ماونتن فيو اي سيتي فيو حديقة الناد...,Cairo,Mountain View iCity,3,3,165,1,3 days ago,resale,...,"[Security, Covered Parking, Maids Room, Pets A...",Descriptionماونتن فيو اي سيتيحديقة النادي شقةا...,Sq. M.,EGP,3,17,4,featured,5145000,5145000
1049,199756521,شقة للبيع 154م بالقرب من AUC في Hap Town حسن ع...,Cairo,Haptown,3,3,154,None,3 days ago,primary,...,"[Balcony, Security, Covered Parking, Maids Roo...",Descriptionشقة للبيع 154م في Hap Town المستقبل...,Sq. M.,EGP,3,3,None,None,800000,800000
1050,199395362,شقه للبيع فورى متشطبه و جاهزه للسكن امام الجام...,Cairo,5th Settlement,3,2,142,None,3 days ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقه ارضى بجاردن متشطبه بالكامل امام...,Sq. M.,EGP,3,6,1,featured,1500000,1500000
1051,199082098,شقه استلام فورى متشطبه ( جاهزه للسكن حالا ),Cairo,New Capital City,3,2,130,None,2 months ago,primary,...,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionكمبوند من هيئه المجتمعات العمرانيه ...,Sq. M.,EGP,3,6,1,None,300000,300000


In [41]:
property_df[property_df.Furnished=='1']

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,Seller Type,amenities_list,description_text,Price
28,198804722,دوبلكس ( مجهزبالكامل Serviced Apartment ) ابرا...,Giza,Sheikh Zayed,5,5,415,1,17 minutes ago,primary,business,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionتطرح شركة اورا للتنمية العقارية ORA...,7400000
59,199787580,ِشقة للبيع متشطبة 3 غرف كمبوند Address East با...,Cairo,The Address East,3,2,180,1,33 minutes ago,primary,business,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقة للبيع في التجمع الخامسبـ كمبوند...,1784000


In [44]:
property_df[property_df.Price<1_000_000]

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,Seller Type,amenities_list,description_text,Price
4,199436704,Apartment for sale in Sunrise near the World M...,Cairo,Shorouk City,3,2,255,None,13 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pool, Ele...",DescriptionApartment for sale in Al-Burj Al-Sh...,900000
5,199138451,شقة للبيع 111م (غرفتين نوم ) فى كمبوند تاج سيت...,Cairo,New Cairo,2,2,111,None,17 minutes ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقة غرفتين نوم للبيع في تاج سيتي - ...,580000
7,199072198,أقل سعر _ شقة بكمبوند Taj City القاهرة الجديدة...,Cairo,Nasr City,2,2,110,2,21 minutes ago,primary,business,"[Balcony, Security, Covered Parking]",Descriptionشقة لقطة للبيع كمبوند تاج سيتى Taj ...,570000
11,199787659,شقة140م إستلام خلال أشهر أمام طريق السويس القا...,Cairo,1st Settlement,2,2,140,None,22 minutes ago,primary,business,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقة 140م للبيع في القاهرة الجديدة ك...,769500
12,199057622,شقة لقطة فى كمبوند Sarai New Cairo سراى - سور ...,Cairo,New Cairo,2,2,111,2,21 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pets Allo...",Descriptionشقة للبيع فى كمبوند سراى القاهرة ال...,300000
13,185506857,شقه (غرفتين ) بالقسط للبيع بدون فوائد القاهره ...,Cairo,Shorouk City,2,2,135,2,3 weeks ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقه135م(غرفتين )بالقسط للبيع بدون ف...,550000
18,199468751,شالية للبيع 130 م تشطيب سوبر لوكس بالتقسيط,Suez,Ain Sukhna,3,2,130,None,18 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pets Allo...",Descriptionشالية 130م للبيع في موقع مميز في ال...,550000
30,199787642,امتلك شقة غرفتين ب اميز سعر فالتجمع و مقدم يبد...,Cairo,5th Settlement,2,2,120,2,25 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Electrici...",Descriptionامتلك شقةغرفتين فالتجمع الخامس في ل...,230000
32,199774986,استلم واسكن فوراً في شقة بڤيو مميز في حدائق أك...,Giza,Sun Capital,2,2,105,2,16 minutes ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقق استلام فوري في المنطقة السياحية...,400000
36,500349836,Apartment 3beds for sale with installment on 8...,Cairo,Hyde Park New Cairo,3,3,206,None,1 hour ago,primary,business,"[Private Garden, Security, Covered Parking, Pe...",DescriptionApartment for sale with installment...,477523


In [55]:
property_df[(property_df.Price<1_000_000)&((property_df['Ad Title'].str.contains('|'.join(['مقدم','قسط', 'installment']))|(property_df['description_text'].str.contains('|'.join(['مقدم','قسط', 'installment'])))]

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (<ipython-input-55-ab35b870d4e1>, line 1)

In [64]:
property_df[((property_df.Price < 1_000_000) & ((property_df['Ad Title'].str.contains('|'.join(['مقدم', 'قسط', 'installment'])) | (property_df['description_text'].str.contains('|'.join(['مقدم', 'قسط', 'installment']))))))]

,Ad ID,Ad Title,Country,Location,Number of Rooms,Number of Bathrooms,Area,Furnished,creation_date,Ownership,Seller Type,amenities_list,description_text,Price
4,199436704,Apartment for sale in Sunrise near the World M...,Cairo,Shorouk City,3,2,255,None,13 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pool, Ele...",DescriptionApartment for sale in Al-Burj Al-Sh...,900000
5,199138451,شقة للبيع 111م (غرفتين نوم ) فى كمبوند تاج سيت...,Cairo,New Cairo,2,2,111,None,17 minutes ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقة غرفتين نوم للبيع في تاج سيتي - ...,580000
7,199072198,أقل سعر _ شقة بكمبوند Taj City القاهرة الجديدة...,Cairo,Nasr City,2,2,110,2,21 minutes ago,primary,business,"[Balcony, Security, Covered Parking]",Descriptionشقة لقطة للبيع كمبوند تاج سيتى Taj ...,570000
11,199787659,شقة140م إستلام خلال أشهر أمام طريق السويس القا...,Cairo,1st Settlement,2,2,140,None,22 minutes ago,primary,business,"[Balcony, Built in Kitchen Appliances, Private...",Descriptionشقة 140م للبيع في القاهرة الجديدة ك...,769500
12,199057622,شقة لقطة فى كمبوند Sarai New Cairo سراى - سور ...,Cairo,New Cairo,2,2,111,2,21 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pets Allo...",Descriptionشقة للبيع فى كمبوند سراى القاهرة ال...,300000
13,185506857,شقه (غرفتين ) بالقسط للبيع بدون فوائد القاهره ...,Cairo,Shorouk City,2,2,135,2,3 weeks ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقه135م(غرفتين )بالقسط للبيع بدون ف...,550000
18,199468751,شالية للبيع 130 م تشطيب سوبر لوكس بالتقسيط,Suez,Ain Sukhna,3,2,130,None,18 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Pets Allo...",Descriptionشالية 130م للبيع في موقع مميز في ال...,550000
30,199787642,امتلك شقة غرفتين ب اميز سعر فالتجمع و مقدم يبد...,Cairo,5th Settlement,2,2,120,2,25 minutes ago,primary,business,"[Balcony, Security, Covered Parking, Electrici...",Descriptionامتلك شقةغرفتين فالتجمع الخامس في ل...,230000
32,199774986,استلم واسكن فوراً في شقة بڤيو مميز في حدائق أك...,Giza,Sun Capital,2,2,105,2,16 minutes ago,primary,business,"[Balcony, Central A/C & heating, Security, Cov...",Descriptionشقق استلام فوري في المنطقة السياحية...,400000
36,500349836,Apartment 3beds for sale with installment on 8...,Cairo,Hyde Park New Cairo,3,3,206,None,1 hour ago,primary,business,"[Private Garden, Security, Covered Parking, Pe...",DescriptionApartment for sale with installment...,477523


In [67]:
property_df2 = property_df.copy()